In [1]:
import logging

import numpy as np
import optimistix as optx

from atmodeller import (
    InteriorAtmosphere,
    Planet,
    SolverParameters,
    Species,
    SpeciesCollection,
    debug_logger,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

logger = debug_logger()
logger.setLevel(logging.INFO)
# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

Atmodeller initialized with double precision (float64)


# Sub-Neptune (e.g. K2-18b) models from Bower et al. (2025)

The code blocks below must always be run, but then you can preferentially run only the models for ideal or real gases.

Parameters for the simulations

In [2]:
number_of_realisations = 500
surface_temperature = 3000.0  # K

# For simulations with fixed mass and surface radius:
planet_mass = 5.154e25
surface_radius = 1.1225e7  # using M-R relation from Hakim+2018
mantle_melt_fraction = 1.0  # 0.1

RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)

In [3]:
solubility_models = get_solubility_models()

H2O_g = Species.create_gas("H2O", solubility=solubility_models["H2O_basalt_dixon95"])
H2_g = Species.create_gas("H2", solubility=solubility_models["H2_basalt_hirschmann12"])
O2_g = Species.create_gas("O2")
CO_g = Species.create_gas("CO", solubility=solubility_models["CO_basalt_yoshioka19"])
CO2_g = Species.create_gas("CO2", solubility=solubility_models["CO2_basalt_dixon95"])
CH4_g = Species.create_gas("CH4", solubility=solubility_models["CH4_basalt_ardia13"])

idealspecies_withsols = SpeciesCollection((H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g))

In [4]:
eos_models = get_eos_models()

H2O_rg = Species.create_gas(
    "H2O",
    activity=eos_models["H2O_cork_holland98"],
    solubility=solubility_models["H2O_basalt_dixon95"],
)
H2_rg = Species.create_gas(
    "H2",
    activity=eos_models["H2_chabrier21"],
    solubility=solubility_models["H2_basalt_hirschmann12"],
)
O2_rg = Species.create_gas("O2")
CO_rg = Species.create_gas(
    "CO",
    activity=eos_models["CO_cs_shi92"],
    solubility=solubility_models["CO_basalt_yoshioka19"],
)
CO2_rg = Species.create_gas(
    "CO2",
    activity=eos_models["CO2_cs_shi92"],
    solubility=solubility_models["CO2_basalt_dixon95"],
)
CH4_rg = Species.create_gas(
    "CH4",
    activity=eos_models["CH4_cs_shi92"],
    solubility=solubility_models["CH4_basalt_ardia13"],
)

realspecies_withsols = SpeciesCollection((H2O_rg, H2_rg, O2_rg, CO_rg, CO2_rg, CH4_rg))

## Vary Hydrogen Mass Fraction

Hydrogen mass fraction varies from 0.1 to 3% of K2-18b's mass

In [5]:
# Vary Linearly:

log10_H_frac = np.linspace(-1.0, 0.5, number_of_realisations)  # 0.1 to 3% of planet mass

# Fix Values for Linearly Varying Cases:
log10_ch_ratios = np.full(number_of_realisations, -0.5)  # 100X Solar
fO2_log10_shifts = np.full(number_of_realisations, -3)

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
}

In [6]:
sub_neptune = Planet(
    surface_temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubilities

In [7]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyHF_withsol = InteriorAtmosphere(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyHF_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyHF_withsol = model_ideal_varyHF_withsol.output

# Write the complete solution to Excel
output_ideal_varyHF_withsol.to_excel("sub_neptune_ideal_withsol_varyHF")

# Write the data to a pickle file with dataframes
output_ideal_varyHF_withsol.to_pickle("sub_neptune_ideal_withsol_varyHF")

[19:54:38 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, SolubilityPowerLaw', 'H2_g: IdealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: IdealGas, _CO_basalt_yoshioka19', 'CO2_g: IdealGas, _CO2_basalt_dixon95', 'CH4_g: IdealGas, _CH4_basalt_ardia13')
[19:54:38 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:54:38 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:54:38 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:54:42 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:54:42 - atmodeller.classes             - INFO     ] - Solver steps (max) = 41
[19:54:42 - atmodeller.output              - INFO     ] - Writing output to excel
[19:54:42 -

### Real gas with solubility

In [8]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyHF_withsol = InteriorAtmosphere(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyHF_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_density=output_ideal_varyHF_withsol.log_number_density,
    solver_parameters=solver_parameters,
)
output_real_varyHF_withsol = model_real_varyHF_withsol.output

# Write the complete solution to Excel
output_real_varyHF_withsol.to_excel("sub_neptune_real_withsol_varyHF")

# Write the data to a pickle file with dataframes
output_real_varyHF_withsol.to_pickle("sub_neptune_real_withsol_varyHF")

[19:54:45 - atmodeller.classes             - INFO     ] - species = ('H2O_g: CombinedRealGas, SolubilityPowerLaw', 'H2_g: CombinedRealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: CombinedRealGas, _CO_basalt_yoshioka19', 'CO2_g: CombinedRealGas, _CO2_basalt_dixon95', 'CH4_g: CombinedRealGas, _CH4_basalt_ardia13')
[19:54:45 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:54:45 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:54:45 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:55:10 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:55:10 - atmodeller.classes             - INFO     ] - Solver steps (max) = 30
[19:55:10 - atmodeller.output              - INFO     ] - 

## Vary Oxygen Fugacity

fO2 varies from IW-6 to IW

In [9]:
# Vary Linearly:
fO2_log10_shifts = np.linspace(-6, 0, number_of_realisations)  # IW-6 to IW


# Fix Values for Linearly Varying Cases:
log10_H_frac = np.full(number_of_realisations, 0)  # 1% of planet mass
log10_ch_ratios = np.full(number_of_realisations, -0.5)  # 100X Solar

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
}

In [10]:
sub_neptune = Planet(
    surface_temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubility

In [11]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyfO2_withsol = InteriorAtmosphere(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyfO2_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyfO2_withsol = model_ideal_varyfO2_withsol.output

# Write the complete solution to Excel
output_ideal_varyfO2_withsol.to_excel("sub_neptune_ideal_withsol_varyfO2")

# Write the data to a pickle file with dataframes
output_ideal_varyfO2_withsol.to_pickle("sub_neptune_ideal_withsol_varyfO2")

[19:55:12 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, SolubilityPowerLaw', 'H2_g: IdealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: IdealGas, _CO_basalt_yoshioka19', 'CO2_g: IdealGas, _CO2_basalt_dixon95', 'CH4_g: IdealGas, _CH4_basalt_ardia13')
[19:55:12 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:55:12 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:55:12 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:55:12 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:55:12 - atmodeller.classes             - INFO     ] - Solver steps (max) = 41
[19:55:12 - atmodeller.output              - INFO     ] - Writing output to excel
[19:55:12 -

### Real gas with solubility

In [12]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyfO2_withsol = InteriorAtmosphere(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyfO2_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_density=output_ideal_varyfO2_withsol.log_number_density,
)
output_real_varyfO2_withsol = model_real_varyfO2_withsol.output

# Write the complete solution to Excel
output_real_varyfO2_withsol.to_excel("sub_neptune_real_withsol_varyfO2")

# Write the data to a pickle file with dataframes
output_real_varyfO2_withsol.to_pickle("sub_neptune_real_withsol_varyfO2")

[19:55:13 - atmodeller.classes             - INFO     ] - species = ('H2O_g: CombinedRealGas, SolubilityPowerLaw', 'H2_g: CombinedRealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: CombinedRealGas, _CO_basalt_yoshioka19', 'CO2_g: CombinedRealGas, _CO2_basalt_dixon95', 'CH4_g: CombinedRealGas, _CH4_basalt_ardia13')
[19:55:13 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:55:13 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:55:13 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:55:15 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:55:15 - atmodeller.classes             - INFO     ] - Solver steps (max) = 21
[19:55:15 - atmodeller.output              - INFO     ] - 

## Vary C/H Ratio

C/H ratio varies from that of solar (log10(C/H) = -2.5) to bulk silicate Earth (log10(C/H)=0.1)

In [13]:
# Vary Linearly:
log10_ch_ratios = np.linspace(-2.5, 0.1, number_of_realisations)  # Solar to BSE Ratios


# Fix Values for Linearly Varying Cases:
log10_H_frac = np.full(number_of_realisations, 0)  # 1% of planet mass
fO2_log10_shifts = np.full(number_of_realisations, -3)

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
}

In [14]:
sub_neptune = Planet(
    surface_temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubility

In [15]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyCtoH_withsol = InteriorAtmosphere(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyCtoH_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyCtoH_withsol = model_ideal_varyCtoH_withsol.output

# Write the complete solution to Excel
output_ideal_varyCtoH_withsol.to_excel("sub_neptune_ideal_withsol_varyCtoH")

# Write the data to a pickle file with dataframes
output_ideal_varyCtoH_withsol.to_pickle("sub_neptune_ideal_withsol_varyCtoH")

[19:55:17 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, SolubilityPowerLaw', 'H2_g: IdealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: IdealGas, _CO_basalt_yoshioka19', 'CO2_g: IdealGas, _CO2_basalt_dixon95', 'CH4_g: IdealGas, _CH4_basalt_ardia13')
[19:55:17 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:55:17 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:55:17 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:55:17 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:55:17 - atmodeller.classes             - INFO     ] - Solver steps (max) = 36
[19:55:17 - atmodeller.output              - INFO     ] - Writing output to excel
[19:55:17 -

### Real gas with solubility

In [16]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyCtoH_withsol = InteriorAtmosphere(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyCtoH_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_density=output_ideal_varyCtoH_withsol.log_number_density,
)
output_real_varyCtoH_withsol = model_real_varyCtoH_withsol.output

# Write the complete solution to Excel
output_real_varyCtoH_withsol.to_excel("sub_neptune_real_withsol_varyCtoH")

# Write the data to a pickle file with dataframes
output_real_varyCtoH_withsol.to_pickle("sub_neptune_real_withsol_varyCtoH")

[19:55:18 - atmodeller.classes             - INFO     ] - species = ('H2O_g: CombinedRealGas, SolubilityPowerLaw', 'H2_g: CombinedRealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: CombinedRealGas, _CO_basalt_yoshioka19', 'CO2_g: CombinedRealGas, _CO2_basalt_dixon95', 'CH4_g: CombinedRealGas, _CH4_basalt_ardia13')
[19:55:18 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:55:18 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:55:18 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:55:20 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:55:20 - atmodeller.classes             - INFO     ] - Solver steps (max) = 18
[19:55:20 - atmodeller.output              - INFO     ] - 

## Vary Planetary Surface Radius

Surface radius varies from 1.76 to 2.6 REarth, planet mass is fixed at 8.63 MEarth

In [17]:
# For simulations with varying surface radius:
surface_radius = np.linspace(1.1225e7, 1.6647e7, number_of_realisations)  # Vary linearly

# Fix Values for Linearly Varying Cases:
log10_H_frac = np.full(
    number_of_realisations, 0.5
)  # ~3% of planet mass, used for fix Surf Radius and Planet Mass cases
log10_ch_ratios = np.full(number_of_realisations, -0.5)  # 100X Solar
fO2_log10_shifts = np.full(number_of_realisations, -3)

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
}

In [18]:
sub_neptune = Planet(
    surface_temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubility

In [19]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyRsurf_withsol = InteriorAtmosphere(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyRsurf_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyRsurf_withsol = model_ideal_varyRsurf_withsol.output

# Write the complete solution to Excel
output_ideal_varyRsurf_withsol.to_excel("sub_neptune_ideal_withsol_varyRsurf")

# Write the data to a pickle file with dataframes
output_ideal_varyRsurf_withsol.to_pickle("sub_neptune_ideal_withsol_varyRsurf")

[19:55:21 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, SolubilityPowerLaw', 'H2_g: IdealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: IdealGas, _CO_basalt_yoshioka19', 'CO2_g: IdealGas, _CO2_basalt_dixon95', 'CH4_g: IdealGas, _CH4_basalt_ardia13')
[19:55:21 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:55:21 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:55:21 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:55:23 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:55:23 - atmodeller.classes             - INFO     ] - Solver steps (max) = 41
[19:55:23 - atmodeller.output              - INFO     ] - Writing output to excel
[19:55:23 -

### Real gas with solubility

In [20]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyRsurf_withsol = InteriorAtmosphere(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyRsurf_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_density=output_ideal_varyRsurf_withsol.log_number_density,
)
output_real_varyRsurf_withsol = model_real_varyRsurf_withsol.output

# Write the complete solution to Excel
output_real_varyRsurf_withsol.to_excel("sub_neptune_real_withsol_varyRsurf")

# Write the data to a pickle file with dataframes
output_real_varyRsurf_withsol.to_pickle("sub_neptune_real_withsol_varyRsurf")

[19:55:24 - atmodeller.classes             - INFO     ] - species = ('H2O_g: CombinedRealGas, SolubilityPowerLaw', 'H2_g: CombinedRealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: CombinedRealGas, _CO_basalt_yoshioka19', 'CO2_g: CombinedRealGas, _CO2_basalt_dixon95', 'CH4_g: CombinedRealGas, _CH4_basalt_ardia13')
[19:55:24 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:55:24 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:55:24 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:55:46 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:55:46 - atmodeller.classes             - INFO     ] - Solver steps (max) = 30
[19:55:46 - atmodeller.output              - INFO     ] - 

## Vary Planetary Mass

Planet mass varies from 4 to 9 MEarth, surface radius is fixed at 1.76 REarth

In [21]:
surface_radius = 1.1225e7  # using M-R relation from Hakim+2018


# For simulations with varying planet mass:

planet_mass_Earths = np.linspace(4, 9, number_of_realisations)  # Vary linearly from 4-9 MEarth
planet_mass = planet_mass_Earths * 5.9722e24


# Fix Values for Linearly Varying Cases:
log10_H_frac = np.full(
    number_of_realisations, 0.5
)  # ~3% of planet mass, used for fix Surf Radius and Planet Mass cases
log10_ch_ratios = np.full(number_of_realisations, -0.5)  # 100X Solar
fO2_log10_shifts = np.full(number_of_realisations, -3)

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
}

In [22]:
sub_neptune = Planet(
    surface_temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubility

In [23]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyMp_withsol = InteriorAtmosphere(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyMp_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyMp_withsol = model_ideal_varyMp_withsol.output

# Write the complete solution to Excel
output_ideal_varyMp_withsol.to_excel("sub_neptune_ideal_withsol_varyMp")

# Write the data to a pickle file with dataframes
output_ideal_varyMp_withsol.to_pickle("sub_neptune_ideal_withsol_varyMp")

[19:55:47 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, SolubilityPowerLaw', 'H2_g: IdealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: IdealGas, _CO_basalt_yoshioka19', 'CO2_g: IdealGas, _CO2_basalt_dixon95', 'CH4_g: IdealGas, _CH4_basalt_ardia13')
[19:55:47 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:55:47 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:55:47 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:55:50 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:55:50 - atmodeller.classes             - INFO     ] - Solver steps (max) = 41
[19:55:50 - atmodeller.output              - INFO     ] - Writing output to excel
[19:55:50 -

### Real gas with solubility

In [24]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyMp_withsol = InteriorAtmosphere(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyMp_withsol.solve(
    planet=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_density=output_ideal_varyMp_withsol.log_number_density,
)
output_real_varyMp_withsol = model_real_varyMp_withsol.output

# Write the complete solution to Excel
output_real_varyMp_withsol.to_excel("sub_neptune_real_withsol_varyMp")

# Write the data to a pickle file with dataframes
output_real_varyMp_withsol.to_pickle("sub_neptune_real_withsol_varyMp")

[19:55:51 - atmodeller.classes             - INFO     ] - species = ('H2O_g: CombinedRealGas, SolubilityPowerLaw', 'H2_g: CombinedRealGas, SolubilityPowerLawLog10', 'O2_g: IdealGas, NoSolubility', 'CO_g: CombinedRealGas, _CO_basalt_yoshioka19', 'CO2_g: CombinedRealGas, _CO2_basalt_dixon95', 'CH4_g: CombinedRealGas, _CH4_basalt_ardia13')
[19:55:51 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 200 K and 6000 K
[19:55:51 - atmodeller.classes             - INFO     ] - reactions = {0: '1.0 H2_g + 0.5 O2_g = 1.0 H2O_g',
 1: '0.5 O2_g + 1.0 CO_g = 1.0 CO2_g',
 2: '2.0 H2_g + 1.0 CO_g = 0.5 O2_g + 1.0 CH4_g'}
[19:55:51 - atmodeller.classes             - INFO     ] - Attempting to solve 500 model(s)
[19:56:13 - atmodeller.classes             - INFO     ] - Solve complete: 500 (100.00%) successful model(s)
[19:56:13 - atmodeller.classes             - INFO     ] - Solver steps (max) = 30
[19:56:13 - atmodeller.output              - INFO     ] - 